# Spylon kernel usage

* Spylon-kernel shares the same spark runtime between Spark(scala) and PySpark

### The cells are by default running scala

In [ ]:
spark

In [ ]:
println("hello")

### We may hop into the python runtime by using the %%python magic keyword in the beginning of a cell

In [ ]:
%%python
import sys
print(sys.version)
spark

We may import source code from any folder in the repository (I prefer to store the code in the source folder)
* Scala code goes into src/scala
* Python code goes into stc/python

## Plotting with python

#### Import the plotfig magic function from helpers
Spylon-kernel needs some workarounds to enable the drawing of figures using the python runtime. Specifically, visualizations need to be output to a temporary file, and an image object containing the temporary file must be passed as a return argument to the kernel. This is all packed into the plotfig_magic() function.


In [ ]:
%%python
import tempfile
from IPython.display import Image
def plotfig_magic():
    fo = tempfile.NamedTemporaryFile(suffix=".png", mode="w")
    fo.close()
    plt.savefig(fo.name)
    return Image(filename=fo.name)

____
##### Replace the use of plt.show() with retval=plotfig_magic

In [ ]:
%%python
#Import helper files from src folder
from src.python.helpers import plotfig_magic

import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
import tempfile

plt.clf()
ax = plt.plot([1,2,3], [3,2,5])
retval = plotfig_magic()

## Combining the use of scala and python

Example: Create dataset with scala (usually we would load it from some source and transform it)

In [ ]:
import spark.implicits._

val values = List((1,1),(2,2),(4,3),(8,4),(16,5),(32,6))
val df = spark.createDataFrame(values).toDF("x", "y")
df.printSchema
df.show

### Register the dataframe as a table in the spark session

In [ ]:
df.createOrReplaceTempView("exampletable")
spark.sql("show tables").show

### Now open it using python's spark session

In [ ]:
%%python
spark.sql("show tables").show()
exampletable = spark.sql("select * from exampletable")
exampletable.show()

### Let's plot the data contained within the table

In [ ]:
%%python
df = exampletable.toPandas()
print(df)
plt.clf()
df.plot(x="x",y="y")
retval=plotfig_magic()